In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import os
from os.path import join
import re
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from functools import reduce
from operator import add
from nltk.corpus import stopwords
import numpy as np



In [2]:
class TextNormalize(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # Replacing the new line symbols wiht space.
        X = X.replace('\n', ' ')
        
        X = X.replace('\r', ' ')
        
        # Bringing the text to the lower case.
        X = X.lower()
        
        # Extracting only the words from the text.
        X = ' '.join(re.findall('[a-z]+', X))
        
        return X
            

NameError: name 'BaseEstimator' is not defined

In [ ]:
text = 'The smooth muscle ﬁbres taper at both ends (fusiform) and do not\nshow striations (Figure 7.7b). Cell junctions hold them together and they\nare bundled together in a connective tissue sheath. The wall of internal\norgans such as the blood vessels, stomach and intestine contains this type\nof muscle tissue. Smooth muscles are ‘involuntary’ as their functioning\ncannot be directly controlled. We usually are not able to make it contract\nmerely by thinking about it as we can do with skeletal muscles.\n\nCardiac muscle tissue is a contractile tissue present only in the heart.\nCell junctions fuse the plasma membranes of cardiac muscle cells and\nmake them stick together (Figure 7.70). Communication junctions\n(intercalated discs) at some fusion points allow the cells to contract as a\nunit, i.e., when one cell receives a signal to contract, its neighbours are\nalso stimulated to contract.'
text

In [ ]:
obj = TextNormalize()
obj.transform(text)

In [ ]:
class WordExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, stop_words = None):
        self.__stop_words = stop_words
    def fit(self, X, y=None):
        self.__hapaxes = []
        
        fdist = FreqDist()
        
        self.__hapaxes = fdist.hapaxes()
        return self
    def transform(self, X, y=None):
        
        self.fit(X)
        self.common_hapaxes = list(reduce(add, [self.__hapaxes]))
        X = ' '.join([word for word in word_tokenize(X)
                                        if word not in self.common_hapaxes])
        
        if self.__stop_words is not None:
            X = ' '.join([word for word in word_tokenize(X)
                            if word not in self.__stop_words])
        return X

In [ ]:
obj2 = WordExtractor(stopwords.words('english'))
obj2.transform(text)

In [ ]:
class ApplyStemmer(BaseEstimator, TransformerMixin):
    def __init__(self, stemmer = None):
        self.stemmer = stemmer
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X =  ' '.join([self.stemmer.stem(word) for word in word_tokenize(X)])
        return X

In [ ]:
obj3 = ApplyStemmer(PorterStemmer())
obj3.transform(text)